get midstream qiita file

run unoise, deblur w positive filter, dada2  
    what do each of these take as input?
        unoise
            preprocessed fasta
            quality filter
            pool samples
            dereplicate
            discard singletons
            unoise
run organelle annotation comparison pipeline

In [59]:
import os
import shutil
import gzip
import urllib.request
from qiime2 import Artifact
from qiime2 import Metadata
from qiime2.plugins.cutadapt.methods import demux_single
from qiime2.plugins.dada2.methods import denoise_single
from qiime2.plugins.deblur.methods import denoise_16S

In [4]:
working_dir = os.path.abspath('../')
refs_dir = working_dir + '/taxonomy_references'

In [5]:
def download_file(url, local_filepath):
    with urllib.request.urlopen(url) as response, open(local_filepath, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

In [17]:
#get the unoise program
if not os.path.isfile(working_dir + '/usearch.gz'):
    download_file('https://drive5.com/downloads/usearch11.0.667_i86linux32.gz', working_dir + '/usearch.gz')
if not os.path.isfile(working_dir + '/usearch'):
    with gzip.open(working_dir + '/usearch.gz', 'rb') as gz:
        with open(working_dir + '/usearch', 'wb') as usearch:
            shutil.copyfileobj(gz, usearch)

In [30]:
!../usearch -fastq_filter ../input/seqs.fastq -fastq_maxee_rate 0.005 -fastaout ../input/usearch/filtered_reads.fasta

usearch v11.0.667_i86linux32, 4.0Gb RAM (12.3Gb total), 12 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: personal use only

00:00 4.2Mb  FASTQ base 33 for file ../input/seqs.fastq
00:00 38Mb   CPU has 12 cores, defaulting to 10 threads
00:02 90Mb    100.0% Filtering, 96.5% passed
    583305  Reads (583.3k)                  
    562695  Filtered reads (562.7k, 96.5%)


In [31]:
!../usearch -fastx_uniques ../input/usearch/filtered_reads.fasta -fastaout ../input/usearch/uniques.fasta -sizeout -relabel Uniq

usearch v11.0.667_i86linux32, 4.0Gb RAM (12.3Gb total), 12 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: personal use only

00:01 142Mb   100.0% Reading ../input/usearch/filtered_reads.fasta
00:01 108Mb  CPU has 12 cores, defaulting to 10 threads           
00:01 307Mb   100.0% DF
00:01 313Mb  562695 seqs, 93938 uniques, 67785 singletons (72.2%)
00:01 313Mb  Min size 1, median 1, max 4521, avg 5.99
00:01 217Mb   100.0% Writing ../input/usearch/uniques.fasta


In [32]:
!../usearch -sortbysize ../input/usearch/uniques.fasta -fastaout ../input/usearch/sorted_uniques.fasta 

usearch v11.0.667_i86linux32, 4.0Gb RAM (12.3Gb total), 12 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: personal use only

00:00 53Mb    100.0% Reading ../input/usearch/uniques.fasta
00:00 19Mb   Getting sizes                                 
00:00 20Mb   Sorting 93938 sequences
00:00 20Mb    100.0% Writing output


In [33]:
!../usearch -unoise3 ../input/usearch/sorted_uniques.fasta -zotus ../output/unoise_zotus.fasta -tabbedout ../output/unoise3.txt

usearch v11.0.667_i86linux32, 4.0Gb RAM (12.3Gb total), 12 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: personal use only

00:00 53Mb    100.0% Reading ../input/usearch/sorted_uniques.fasta
00:01 21Mb      0.0% 0 amplicons, 0 bad (size >= 4521)            

00:01 55Mb    100.0% 4604 amplicons, 27946 bad (size >= 8)
00:06 62Mb    100.0% 3819 good, 785 chimeras              
00:06 62Mb    100.0% Writing zotus          


In [ ]:
#end of usearch/unoise

In [34]:
#run dada2 on dataset
#can be done in qiime

In [41]:
#import data to qiime
fastq = Artifact.import_data('MultiplexedSingleEndBarcodeInSequence', working_dir + '/input/121714_seqs.fastq.gz')
fastq.save(working_dir + '/input/qiime2/multiplexed-seqs.qza')

'/mnt/c/Users/dsone/Documents/zaneveld/organelle_project/input/qiime2/multiplexed-seqs.qza'

In [48]:
with open(working_dir + '/input/12366_prep_6501_20201201-082351.txt') as file:
    lines = file.readlines()
    lines[0] = 'sample-id\tbarcode\tcenter_name\texperiment_design_description\tinstrument_model\tlibrary_construction_protocol\tpcr_primers\tplatform\tprimer\tqiita_prep_id\tsequencing_meth\ttarget_gene\ttarget_subfragment\n'
    with open(working_dir + '/input/qiime2/barcode_file.tsv', 'w') as metadata:
        for line in lines:
            metadata.write(line)

In [49]:
metadata = Metadata.load(working_dir + '/input/qiime2/barcode_file.tsv')

In [50]:
barcodes = metadata.get_column('barcode')
barcodes

<CategoricalMetadataColumn name='barcode' id_count=105>

In [51]:
demultiplexed_per_sample_sequences, untrimmed_sequences = demux_single(fastq, barcodes, 0)

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: cutadapt --front file:/tmp/tmp0v21hssa --error-rate 0 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-rdpkecgq/{name}.1.fastq.gz --untrimmed-output /tmp/q2-MultiplexedSingleEndBarcodeInSequenceDirFmt-0t7ri8qe/forward.fastq.gz /tmp/qiime2-archive-nx0nmfn6/4321180c-996d-4d99-9cde-393384716461/data/forward.fastq.gz



In [53]:
demultiplexed_per_sample_sequences.save(working_dir + '/input/qiime2/demuxed.qza')

'/mnt/c/Users/dsone/Documents/zaneveld/organelle_project/input/qiime2/demuxed.qza'

In [63]:
#streamlined
#convert prep file to metadata file
#with open(working_dir + '/input/12366_prep_6501_20201201-082351.txt') as file:
#    lines = file.readlines()
#    lines[0] = 'sample-id\tbarcode\tcenter_name\texperiment_design_description\tinstrument_model\tlibrary_construction_protocol\tpcr_primers\tplatform\tprimer\tqiita_prep_id\tsequencing_meth\ttarget_gene\ttarget_subfragment\n'
#    with open(working_dir + '/input/qiime2/barcode_file.tsv', 'w') as metadata:
#        for line in lines:
#            metadata.write(line)
metadata = Metadata.load(working_dir + '/input/qiime2/barcode_file.tsv')
barcodes = metadata.get_column('barcode')

fastq = Artifact.import_data('MultiplexedSingleEndBarcodeInSequence', working_dir + '/input/70146_seqs.fastq.gz')
demultiplexed_per_sample_sequences, untrimmed_sequences = demux_single(fastq, barcodes, 0)


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: cutadapt --front file:/tmp/tmp9zmu50cz --error-rate 0 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-7odkbpyl/{name}.1.fastq.gz --untrimmed-output /tmp/q2-MultiplexedSingleEndBarcodeInSequenceDirFmt-o46lzrbp/forward.fastq.gz /tmp/qiime2-archive-15rxqvz6/97dc9fb5-151e-41db-915d-aea6c531d4ee/data/forward.fastq.gz



In [64]:
#dada2
table, rep_seqs, stats = denoise_single(demultiplexed_per_sample_sequences, 100, n_threads = 0)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_single.R /tmp/qiime2-archive-3jycpggi/8edd41ed-b35c-40bd-8b30-ac4991d16d09/data /tmp/tmp5wypwrmi/output.tsv.biom /tmp/tmp5wypwrmi/track.tsv /tmp/tmp5wypwrmi 100 0 2.0 2 Inf independent consensus 1.0 0 1000000 NULL 16



In [65]:
table.save(working_dir + '/output/dada2_table.qza')
rep_seqs.save(working_dir + '/output/dada2_rep_seqs.qza')

'/mnt/c/Users/dsone/Documents/zaneveld/organelle_project/output/dada2_rep_seqs.qza'

In [66]:
#deblur
table, rep_seqs, stats = denoise_16S(demultiplexed_per_sample_sequences, 100, jobs_to_start = 4)

In [67]:
table.save(working_dir + '/output/deblur_table.qza')
rep_seqs.save(working_dir + '/output/deblur_rep_seqs.qza')

'/mnt/c/Users/dsone/Documents/zaneveld/organelle_project/output/deblur_rep_seqs.qza'